# RNA velocity analysis in zebrafish neutrophils

## Import packages & data

In [ ]:
# load packages needed
import scanpy as sc
import anndata as ad
from scipy import io
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import os
import pandas as pd
import scvelo as scv
import matplotlib.pyplot as plt
# import cellrank as cr

In [ ]:
# load sparse matrix:
X = io.mmread("burn_neuts_counts.mtx")

# create anndata object
adata = ad.AnnData(
    X=X.transpose().tocsr()
)

# load cell metadata:
cell_meta = pd.read_csv("burn_neuts_metadata.csv")

# load gene names:
with open("burn_neuts_gene_names.csv", 'r') as f:
    gene_names = f.read().splitlines()

# set anndata observations and index obs by barcodes, var by gene names
adata.obs = cell_meta
adata.obs.index = adata.obs['barcode']
adata.var.index = gene_names

# load dimensional reduction:
pca = pd.read_csv("burn_neuts_pca.csv")
pca.index = adata.obs.index

# set pca and umap
adata.obsm['X_pca'] = pca.to_numpy()
adata.obsm['X_umap'] = np.vstack((adata.obs['UMAP_1'].to_numpy(), adata.obs['UMAP_2'].to_numpy())).T

In [ ]:
# Process data with velocyto
## setting scVelo basic parameters
scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', facecolor='white', dpi=80,dpi_save=300 ,frameon=False,
                               fontsize=8, transparent=True, vector_friendly=True)

In [ ]:
# Reorder identities
adata.obs['Identity'].cat.reorder_categories(['Precursor-like_Neuts','Immature_Neuts','il6r_padi2_Neuts', 'Mature_Neuts'],
                                            inplace = True)
adata.obs['Identity']

In [ ]:
fig = sc.pl.umap(adata, color=['Identity'], frameon=False,
           palette = ("#666666","#AD7700","#1C91D4","#007756"),return_fig=True)
ax = plt.gca()
ax.set_aspect(1)
ax.set_title('')
# plt.savefig("figures/umap_neuts_only.svg")

In [ ]:
# Load loom file
ldata_3dpf = scv.read('velocyto_looms/WT_3dpf.loom', cache=True)
ldata_4dpf = scv.read('velocyto_looms/WT_4dpf.loom', cache=True)
ldata_5dpf = scv.read('velocyto_looms/WT_5dpf.loom', cache=True)
ldata_6hpb = scv.read('velocyto_looms/WT_6hpb.loom', cache=True)
ldata_24hpb = scv.read('velocyto_looms/WT_24hpb.loom', cache=True)
ldata_48hpb = scv.read('velocyto_looms/WT_48hpb.loom', cache=True)

### Barcode renaming

#### Rename barcodes in order to merge
* ldata_3dpf, ldata_4dpf, ldata_5dpf, ldata_6hpb, ldata_24hpb, ldata_48hpb
* Unwounded_3dpf_, Unwounded_4dpf_, Unwounded_5dpf_, Burn_6hpb_, Burn_24hpb_, Burn_48hpb_

In [ ]:
# Barcode rename for merging
barcodes = [bc.split(':')[1] for bc in ldata_3dpf.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
barcodes = ['Unwounded_3dpf_' + bc for bc in barcodes]
ldata_3dpf.obs.index = barcodes

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata_4dpf.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
barcodes = ['Unwounded_4dpf_' + bc for bc in barcodes]
ldata_4dpf.obs.index = barcodes

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata_5dpf.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
barcodes = ['Unwounded_5dpf_' + bc for bc in barcodes]
ldata_5dpf.obs.index = barcodes

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata_6hpb.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
barcodes = ['Burn_6hpb_' + bc for bc in barcodes]
ldata_6hpb.obs.index = barcodes

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata_24hpb.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
barcodes = ['Burn_24hpb_' + bc for bc in barcodes]
ldata_24hpb.obs.index = barcodes

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata_48hpb.obs.index.tolist()]
barcodes = [bc[0:len(bc)-1] + '-1' for bc in barcodes]
barcodes = ['Burn_48hpb_' + bc for bc in barcodes]
ldata_48hpb.obs.index = barcodes

### Make variables unique

In [ ]:
ldata_3dpf.var_names_make_unique()
ldata_4dpf.var_names_make_unique()
ldata_5dpf.var_names_make_unique()
ldata_6hpb.var_names_make_unique()
ldata_24hpb.var_names_make_unique()
ldata_48hpb.var_names_make_unique()

### Merge loom with anndata

In [ ]:
# concatenate the three loom
ldata = ldata_3dpf.concatenate([ldata_4dpf, ldata_5dpf, ldata_6hpb, ldata_24hpb, ldata_48hpb])
# merge matrices into the original adata object
scv.utils.clean_obs_names(adata)
scv.utils.clean_obs_names(ldata)

neuts = scv.utils.merge(adata, ldata)

In [ ]:
# plot umap to check merging
sc.pl.umap(neuts, color='Identity', frameon=False, legend_loc='on data', title=''
           #,save='_celltypes_neuts_only.pdf'
          )


## scVelo preprocessing

In [ ]:
scv.pl.proportions(neuts, groupby='Identity', save="spliced_unspliced_proportinos_neuts_only.svg")

## Run scVelo

### Split by timepoint and conditions

In [ ]:
# Subsetting adata
# "U_03dpf" "U_04dpf" "U_05dpf" "B_06hpb" "B_24hpb" "B_48hpb"
u_03dpf = neuts[neuts.obs['abbreviation'].isin(["U_03dpf"])]
u_04dpf = neuts[neuts.obs['abbreviation'].isin(["U_04dpf"])]
u_05dpf = neuts[neuts.obs['abbreviation'].isin(["U_05dpf"])]
b_06hpb = neuts[neuts.obs['abbreviation'].isin(["B_06hpb"])]
b_24hpb = neuts[neuts.obs['abbreviation'].isin(["B_24hpb"])]
b_48hpb = neuts[neuts.obs['abbreviation'].isin(["B_48hpb"])]

In [ ]:
# Define a function for running stochastic velocity calculation and plotting
def cal_stochastic_velo(subset, name):
    scv.pp.filter_and_normalize(subset, min_shared_counts=20, n_top_genes=3000)
    sc.tl.pca(subset)
    sc.pp.neighbors(subset, n_pcs=30, n_neighbors=30)
    scv.pp.moments(subset)
    # run stochastic
    scv.tl.velocity(subset, mode="stochastic")
    scv.tl.velocity_graph(subset)
    plt.nms = ["neuts_withVelocity_stochastic_neuts_only_", name, ".svg"]
    plt.nm = "".join(plt.nms)
    scv.pl.velocity_embedding_stream(
        subset, basis="umap", color="Identity", figsize=[1.4,2.1] ,legend_loc="none",
        title="", smooth=0.8, min_mass=2, palette = ("#666666","#AD7700","#1C91D4","#007756"),
        size = 50, arrow_size = 0.5,
        dpi = 300,
        save=plt.nm
    )

In [ ]:
# "U_03dpf" "U_04dpf" "U_05dpf" "B_06hpb" "B_24hpb" "B_48hpb"
cal_stochastic_velo(u_03dpf, "U_03dpf")
cal_stochastic_velo(u_04dpf, "U_04dpf")
cal_stochastic_velo(u_05dpf, "U_05dpf")
cal_stochastic_velo(b_06hpb, "B_06hpb")
cal_stochastic_velo(b_24hpb, "B_24hpb")
cal_stochastic_velo(b_48hpb, "B_48hpb")